In [1]:
from datasets import load_from_disk
from sentence_transformers import SentenceTransformer, CrossEncoder
import numpy as np

In [2]:
device = "mps"

In [3]:
dataset = load_from_disk("outputs/dataset")

In [4]:
dataset

Dataset({
    features: ['file_name', 'text'],
    num_rows: 1163
})

In [5]:
dataset["file_name"]

['3294201.pdf',
 '2896853.pdf',
 '3286177.pdf',
 '3064176.pdf',
 '3291351.pdf',
 '2896890.pdf',
 '3197767.pdf',
 '3014452.pdf',
 '3156113.pdf',
 '3119610.pdf',
 '2916289.pdf',
 '3289734.pdf',
 '3264482.pdf',
 '3218987.pdf',
 '3072258.pdf',
 '2979114.pdf',
 '3011072.pdf',
 '3234504.pdf',
 '2916276.pdf',
 '3250900.pdf',
 '3137923.pdf',
 '2916510.pdf',
 '2977703.pdf',
 '3219509.pdf',
 '2950394.pdf',
 '3250848.pdf',
 '2977930.pdf',
 '3003258.pdf',
 '3214950.pdf',
 '3009007.pdf',
 '3250860.pdf',
 '3015823.pdf',
 '3015837.pdf',
 '3015604.pdf',
 '3013249.pdf',
 '3119016.pdf',
 '3189404.pdf',
 '2937859.pdf',
 '3076375.pdf',
 '3015822.pdf',
 '2977528.pdf',
 '2926997.pdf',
 '2977925.pdf',
 '3072305.pdf',
 '3175456.pdf',
 '3174990.pdf',
 '2938001.pdf',
 '3233242.pdf',
 '3265004.pdf',
 '3137922.pdf',
 '3195415.pdf',
 '2951114.pdf',
 '3267149.pdf',
 '3030056.pdf',
 '3072271.pdf',
 '3289709.pdf',
 '3011932.pdf',
 '3016536.pdf',
 '2974182.pdf',
 '3154705.pdf',
 '2896661.pdf',
 '3189216.pdf',
 '301577

In [6]:
model = SentenceTransformer("sentence-transformers/multi-qa-mpnet-base-dot-v1")
model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [7]:
sum(p.numel() for p in model.parameters())

109486464

In [8]:
# get number of parameters
sum(p.numel() for p in model.parameters() if p.requires_grad)

109486464

In [ ]:
# dataset = dataset.select(range(10))

In [10]:
embeddings = model.encode(dataset["text"], convert_to_tensor=True)
embeddings.shape

torch.Size([10, 768])

In [11]:
dataset = dataset.add_column("embeddings", embeddings.cpu().tolist())

In [12]:
dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['file_name', 'text', 'embeddings'],
    num_rows: 10
})

In [13]:
dataset[0]

{'file_name': '3294201.pdf',
 'text': 'ZARZĄDZENIE NR 683/2024\nBURMISTRZA BRZEGU\nz dnia 27 grudnia 2024 r.\nw sprawie: zmian w planie finansowym budżetu Gminy Brzeg i Urzędu Miasta\nna 2024 rok oraz w planie finansowym MOPS w Brzegu na 2024 rok\nNa podstawie art. 30 ust. 2 pkt 4 ustawy z dnia 8 marca 1990 r. o samorządzie\ngminnym (Dz. U. z 2024 r. poz. 1465, poz. 1572) zarządzam, co następuje:\n§ 1\n1. Zwiększyć plan dochodów bieżących budżetu gminy o kwotę 40,00 zł\nrealizowany przez Urząd Miasta\nzadania zlecone\nw dz. 852 rozdz. 85278 § 2010 o kwotę 40,00 zł\n2. Zwiększyć plan wydatków bieżących budżetu gminy o kwotę 40,00 zł\nrealizowany przez Miejski Ośrodek Pomocy Społecznej\nzadania zlecone\nw dz. 852 rozdz. 85278 § 4010 o kwotę 33,00 zł\n§ 4110 o kwotę 6,00 zł\n§ 4120 o kwotę 1,00 zł\nrazem 40,00 zł\n§ 2\nZarządzenie wchodzi w życie z dniem podpisania.\nBurmistrz\n(-) Violetta Jaskólska-Palus\nUzasadnienie\nZmiany w budżecie gminy na 2024 rok wprowadza się na podstawie\nzawi

In [27]:
def retrieve(query, top_k=50, dataset=dataset, model=model):
    query_embedding = model.encode(query)

    _, samples = dataset.get_nearest_examples(
        "embeddings", query_embedding, k=top_k
    )

    return samples

In [28]:
samples = retrieve("Jaki sklad ma Komisja Egzaminacyjnej od egaminu służby przygotowawczej?")

In [33]:
len(samples["text"])

10

In [31]:
for i, t in enumerate(samples["text"]):
    print(i, t, "\n")

0 Zarządzenie Nr 249 / 2024
Burmistrza Brzegu
z dnia 7 sierpnia 2024 roku
w sprawie: powołania Komisji Przetargowej do przeprowadzenia postępowania o
udzielenie zamówienia publicznego
Na podstawie art. 53 ust. 2 i art. 55 ust. 1 ustawy z dnia 11 września 2019
roku Prawo zamówień publicznych (Dz. U. z 2023 r. poz. 1605 z późn. zm.) oraz § 18
Załącznika nr 1 do Zarządzenia Nr 3626 /2023 Burmistrza Brzegu z dnia 20 stycznia
2023 roku w sprawie nadania Regulaminu udzielania zamówień publicznych przez
Gminę Brzeg prowadzonych przez Urząd Miasta w Brzegu
zarządzam, co następuje:
§ 1
W celu przeprowadzenia postępowania o udzielenie zamówienia klasycznego, o
wartości poniżej progu unijnego, na zadanie pn.: „Grupowe ubezpieczenie na życie
pracowników Urzędu Miasta w Brzegu oraz członków ich rodzin na lata 2024-2026”
OR-IV.271.1.5.2024, powołuję komisję przetargową w następującym składzie:
1) Przewodnicząca: Krystyna Nowakowska – Bider – Sekretarz Brzegu,
2) Z-ca Przewodniczącej: Katarzyna Szcze

In [17]:
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
reranker.model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 384, padding_idx=0)
      (position_embeddings): Embedding(512, 384)
      (token_type_embeddings): Embedding(2, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-5): 6 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
              (LayerNorm): LayerNorm((384,), eps=1e-1

In [ ]:
def rerank(reranker, samples, query, top_k=3):
    reranker_input = [
        [query, text] for text in samples["text"]
    ]   
    
    reranker_scores = reranker.predict(reranker_input)

    ranking = np.argsort(reranker_scores)[::-1]

    top_k_indices = ranking[:top_k]

    ranked_samples = {
        "file_name": [samples["file_name"][i] for i in top_k_indices],
        "text": [samples["text"][i] for i in top_k_indices]
    }

    return ranked_samples

rerank(reranker, samples, "Jaki sklad ma Komisja Egzaminacyjnej od egaminu służby przygotowawczej?")

{'file_name': ['3119610.pdf', '2896890.pdf', '3291351.pdf'],
 'text': ['Zarządzenie Nr 249 / 2024\nBurmistrza Brzegu\nz dnia 7 sierpnia 2024 roku\nw sprawie: powołania Komisji Przetargowej do przeprowadzenia postępowania o\nudzielenie zamówienia publicznego\nNa podstawie art. 53 ust. 2 i art. 55 ust. 1 ustawy z dnia 11 września 2019\nroku Prawo zamówień publicznych (Dz. U. z 2023 r. poz. 1605 z późn. zm.) oraz § 18\nZałącznika nr 1 do Zarządzenia Nr 3626 /2023 Burmistrza Brzegu z dnia 20 stycznia\n2023 roku w sprawie nadania Regulaminu udzielania zamówień publicznych przez\nGminę Brzeg prowadzonych przez Urząd Miasta w Brzegu\nzarządzam, co następuje:\n§ 1\nW celu przeprowadzenia postępowania o udzielenie zamówienia klasycznego, o\nwartości poniżej progu unijnego, na zadanie pn.: „Grupowe ubezpieczenie na życie\npracowników Urzędu Miasta w Brzegu oraz członków ich rodzin na lata 2024-2026”\nOR-IV.271.1.5.2024, powołuję komisję przetargową w następującym składzie:\n1) Przewodnicząca: Kr

In [19]:
# get reranker num of parameters that are trainable
sum(p.numel() for p in reranker.model.parameters() if p.requires_grad)

22713601